<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto City</font></h1>

## Introduction

In this assignment I will be exploring neighbourhoods of Toronto city using Foursquare API. I will be using the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighbourhoods into clusters. I will use the k-means clustering algorithm to cluster the neighbourhoods. The data for the neighbourhoods will be obtained from the wikipidea page **List of neighbourhoods in Toronto**. Finally I will visualize the neihbourhoods and their clusters using the Folium library.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in Toronto City</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
#!pip install BeautifulSoup4

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# Web scrapping
from bs4 import BeautifulSoup
import urllib.request
import csv

print('Libraries imported.')

Libraries imported.


## 1. Download and Explore Dataset

Toronto has a total of 206 neighbourhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the neighborhoods as well as the the latitude and logitude coordinates of each neighborhood. 

The neighbourhoods can be obtained from the wikipedia page **List of neighbourhoods in Toronto** at the link: https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto. the latitude and longitude for each neighbourhood can be obtained by following its link for example latitude/longitude for Grange Park can be obtained from the link: https://en.wikipedia.org/wiki/Grange_Park_(neighbourhood)

Functions used in cleaning and formatting data

In [3]:
#Function to convert lattitude/longitude from degree to float 
def conversion(old):
    direction = {'N':1, 'S':-1, 'E': 1, 'W':-1}
    new = old.replace(u'°',' ').replace('′',' ').replace('″',' ')
    new = new.split()
    new_dir = new.pop()
    new.extend([0,0,0])
    return (float(new[0])+float(new[1])/60.0+float(new[2])/3600.0) * direction[new_dir]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Define the data frame to collect data

In [4]:
# define the dataframe columns
column_names = ['Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Neighborhood,Latitude,Longitude


#### Load and explore the data

Get the raw data

In [5]:
# specify the url
urlpage =  'https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto'

In [6]:
# query the website and return the html to the variable 'page'
page = urllib.request.urlopen(urlpage)
# parse the html using beautiful soup and store in variable 'soup'
soup = BeautifulSoup(page, 'html.parser')

Parse clean and store the data in the dataframe

In [7]:
# Parse and get data

subsecs = soup.find_all('ul')
print('Number of subsecs', len(subsecs))
for i in range(3,21):
    ss = subsecs[i]
    liss = ss.find_all('li')
    #print(liss[2].getText())
    lll =  len(liss)
    #print(i,'] Number of li',lll)
    for j in liss: #(0, lll):
        llu = j.find('a').get('href')
        p2 =  urllib.request.urlopen('https://en.wikipedia.org'+llu)
        s2 = BeautifulSoup(p2, 'html.parser')
        latt='N/A'
        lng='N/A'
        em = ''
        try:
            latt = conversion(s2.find('span', attrs={'class':'latitude'}).getText())
            lng = conversion(s2.find('span', attrs={'class':'longitude'}).getText())
            neighborhoods = neighborhoods.append({'Neighborhood': j.getText(),
                                          'Latitude': latt,
                                          'Longitude': lng}, ignore_index=True)
        except:
            em = "Error getting lat/long"
            
        #flatt = conversion(latt)
        #flng = conversion(lng)
        #print(j.getText(), latt, lng, em) #, flatt, flng)
        
    #print('\r\n')
#results = table.find_all('tr')
#print('Number of results', len(results))

Number of subsecs 79


Let's take a quick look at the data.

In [8]:
print (neighborhoods.shape)
neighborhoods.head()

(206, 3)


,Neighborhood,Latitude,Longitude
0,Alexandra Park,43.650000,-79.400000
1,The Annex,43.670000,-79.403889
2,Baldwin Village,43.656111,-79.393333
3,Cabbagetown,43.666389,-79.362778
4,CityPlace,43.640000,-79.395278


Use geopy library to get the latitude and longitude values of Toronto City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>tt_explorer</em>, as shown below.

In [9]:
address = 'Toronto,ON,Canada'

geolocator = Nominatim(user_agent="tt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


##### create map of Toront city and its neighbourhoods using latitude and longitude values

In [10]:
# create map of Toront city and its neighbourhoods using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    #print(neighborhood, lat,lng)
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#ff9999',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [11]:
CLIENT_ID = '2D2A1WPTXYQD0FPAL2RN3I2FSUZR5JMKOQYOPOHGYBP12NMH' # Foursquare ID
CLIENT_SECRET = '1V1RJ4ANANHR1MV3WQM4NP124KHV455HWJS5TSGVBPOBIE1W' # Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

CLIENT_ID: 2D2A1WPTXYQD0FPAL2RN3I2FSUZR5JMKOQYOPOHGYBP12NMH
CLIENT_SECRET:1V1RJ4ANANHR1MV3WQM4NP124KHV455HWJS5TSGVBPOBIE1W


In [12]:
neighborhoods.loc[0, 'Neighborhood']

'Alexandra Park'

Get the neighborhood's latitude and longitude values.

In [13]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Alexandra Park are 43.65, -79.4.


#### Now, let's get the top 100 venues that are in Alexandra Park within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [14]:
radius=500
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=2D2A1WPTXYQD0FPAL2RN3I2FSUZR5JMKOQYOPOHGYBP12NMH&client_secret=1V1RJ4ANANHR1MV3WQM4NP124KHV455HWJS5TSGVBPOBIE1W&ll=43.65,-79.4&v=20180605&radius=500&limit=100'

Send the GET request and examine the resutls

In [15]:
results = requests.get(url).json()
#results

In [16]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Maker Pizza,Pizza Place,43.650401,-79.398040
1,Saku Sushi,Sushi Restaurant,43.648038,-79.400268
2,Core Studio Yoga & Pilates,Yoga Studio,43.647920,-79.400196
3,Sonic Boom,Record Shop,43.650859,-79.396985
4,BarChef,Bar,43.648038,-79.400161


In [17]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

Since we will be using top 10 venues for clusturing, the function  not add neighbourhoods that have less than 10 venues

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    i = 0;
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        #print(name,'\t', url)
        # make the GET request
        i = i + 1
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']

            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        except:
            print('') #print(name, "Error in getting json", i)
            #del venues_list[-1]
               
        #print(name, len(venues_list[i-1]), i)
        
        if len(venues_list[-1]) < 10:
            print(name,  len(venues_list[-1]), i)
            del venues_list[-1]
            
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [19]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude'],
                                   radius=1000
                                  )


Runnymede 8 82
Clairville 3 99
Markland Wood 9 110
Princess Gardens 4 113
Rexdale 9 114
Thorncrest Village 7 120
West Humber-Clairville 3 121
The Bridle Path 8 131
Downsview 5 135
Humber Summit 7 141
Birch Cliff 8 165
 Birch Cliff Heights 8 166
Highland Creek 4 175
Morningside Heights 5 182
Rouge 4 185
Scarborough Junction 8 187
West Rouge 5 191


In [20]:
toronto_venues.groupby('Neighborhood').count().shape

(186, 6)

In [21]:
print(toronto_venues.shape)
toronto_venues.head()

(10772, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alexandra Park,43.65,-79.4,Maker Pizza,43.650401,-79.398040,Pizza Place
1,Alexandra Park,43.65,-79.4,Sonic Boom,43.650859,-79.396985,Record Shop
2,Alexandra Park,43.65,-79.4,City Dance Corps,43.648144,-79.398111,Dance Studio
3,Alexandra Park,43.65,-79.4,Banh Mi Boys,43.648650,-79.396859,Sandwich Place
4,Alexandra Park,43.65,-79.4,Core Studio Yoga & Pilates,43.647920,-79.400196,Yoga Studio


Let's check how many venues were returned for each neighborhood

In [22]:
toronto_venues.groupby('Neighborhood').count().shape


(186, 6)

#### Let's find out how many unique categories can be curated from all the returned venues

In [23]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 347 uniques categories.


## 3. Analyze Each Neighborhood

In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Motorsports Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sc

And let's examine the new dataframe size.

In [25]:
toronto_onehot.shape

(10772, 347)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [26]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Motorsports Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sc

#### Let's confirm the new size

In [27]:
toronto_grouped.shape

(186, 347)

#### Let's print each neighborhood along with the top 5 most common venues

In [28]:
num_top_venues = 5
 
for hood in toronto_grouped['Neighborhood']:
     print("----"+hood+"----")
     temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
     temp.columns = ['venue','freq']
     temp = temp.iloc[1:]
     temp['freq'] = temp['freq'].astype(float)
     temp = temp.round({'freq': 2})
     print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
     print('\n')

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Food Court,Cantonese Restaurant,Park,Asian Restaurant,Liquor Store,Gas Station,Restaurant,Rental Car Location,Peking Duck Restaurant
1,Alderwood,Hotel,Pharmacy,Pizza Place,Skating Rink,Intersection,Gym,Grocery Store,Gas Station,Moroccan Restaurant,Park
2,Alexandra Park,Coffee Shop,Café,Bar,Italian Restaurant,Vegetarian / Vegan Restaurant,Yoga Studio,French Restaurant,Pizza Place,Gym,Dessert Shop
3,Amesbury,Coffee Shop,Pizza Place,Pharmacy,Vietnamese Restaurant,Trail,Supermarket,Dim Sum Restaurant,Mediterranean Restaurant,Park,Athletics & Sports
4,Armadale,Coffee Shop,Sandwich Place,Grocery Store,Fast Food Restaurant,Chinese Restaurant,Japanese Restaurant,Bakery,Juice Bar,Big Box Store,Gym


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 7 clusters.

In [31]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 2, 0, 4, 3, 0, 2, 0, 0], dtype=int32)

In [32]:
#neighborhoods_venues_sorted.drop(columns=['Cluster Labels'], inplace=True)
#neighborhoods_venues_sorted.columns

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [33]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alexandra Park,43.650000,-79.400000,2.0,Coffee Shop,Café,Bar,Italian Restaurant,Vegetarian / Vegan Restaurant,Yoga Studio,French Restaurant,Pizza Place,Gym,Dessert Shop
1,The Annex,43.670000,-79.403889,3.0,Café,Restaurant,Italian Restaurant,Bakery,Coffee Shop,Vegetarian / Vegan Restaurant,Sushi Restaurant,Pizza Place,Beer Bar,French Restaurant
2,Baldwin Village,43.656111,-79.393333,2.0,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Ramen Restaurant,Tea Room,French Restaurant,Mexican Restaurant,Vietnamese Restaurant,Gastropub
3,Cabbagetown,43.666389,-79.362778,3.0,Coffee Shop,Restaurant,Bakery,Café,Park,Thai Restaurant,Diner,Fast Food Restaurant,Bar,Chinese Restaurant
4,CityPlace,43.640000,-79.395278,3.0,Coffee Shop,Restaurant,Hotel,Italian Restaurant,Park,Gym,Yoga Studio,Baseball Stadium,Dance Studio,Scenic Lookout


In [34]:
#toronto_merged['Cluster Labels'].isna().value_counts()
#remove neighbourhoods that have NA
toronto_merged = toronto_merged[np.isfinite(toronto_merged['Cluster Labels'])]

In [35]:
toronto_merged.isna().describe()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
count,189,189,189,189,189,189,189,189,189,189,189,189,189,189
unique,1,1,1,1,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False,False,False,False,False
freq,189,189,189,189,189,189,189,189,189,189,189,189,189,189


In [36]:
toronto_merged[['Cluster Labels']] = toronto_merged[['Cluster Labels']].astype('int32')
toronto_merged.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alexandra Park,43.650000,-79.400000,2,Coffee Shop,Café,Bar,Italian Restaurant,Vegetarian / Vegan Restaurant,Yoga Studio,French Restaurant,Pizza Place,Gym,Dessert Shop
1,The Annex,43.670000,-79.403889,3,Café,Restaurant,Italian Restaurant,Bakery,Coffee Shop,Vegetarian / Vegan Restaurant,Sushi Restaurant,Pizza Place,Beer Bar,French Restaurant
2,Baldwin Village,43.656111,-79.393333,2,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Ramen Restaurant,Tea Room,French Restaurant,Mexican Restaurant,Vietnamese Restaurant,Gastropub
3,Cabbagetown,43.666389,-79.362778,3,Coffee Shop,Restaurant,Bakery,Café,Park,Thai Restaurant,Diner,Fast Food Restaurant,Bar,Chinese Restaurant
4,CityPlace,43.640000,-79.395278,3,Coffee Shop,Restaurant,Hotel,Italian Restaurant,Park,Gym,Yoga Studio,Baseball Stadium,Dance Studio,Scenic Lookout


In [37]:
toronto_merged['Cluster Labels'].value_counts()

3    76
4    45
0    37
2    23
1     8
Name: Cluster Labels, dtype: int64

##### Finally, let's visualize the resulting clusters

In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters



In [39]:
tm1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]
tm1['1st Most Common Venue'].value_counts()

Park                      17
Coffee Shop                4
Grocery Store              3
Convenience Store          3
Sandwich Place             2
Pizza Place                2
Harbor / Marina            1
Bakery                     1
Bagel Shop                 1
Furniture / Home Store     1
Hotel                      1
Pharmacy                   1
Name: 1st Most Common Venue, dtype: int64

### As can be seen above in the 1st cluster top most 1st most common venue is 'Park' so this cluster can be termed as 'Park' neighbourhoods

In [40]:
tm2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]
tm2['1st Most Common Venue'].value_counts()

Korean Restaurant     6
Chinese Restaurant    2
Name: 1st Most Common Venue, dtype: int64

### As can be seen above in the 2nd cluster 1st most common venue are oriental restaurant so this cluster can be termed as 'Oriental Cuisine' neighbourhoods

In [41]:
tm3 =toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]
tm3['1st Most Common Venue'].value_counts()

Café           13
Bar             6
Coffee Shop     4
Name: 1st Most Common Venue, dtype: int64

### As can be seen above in the 3rd cluster majority of 1st most common venue are Café so this cluster can be termed as 'Café' neighbourhoods

In [42]:
tm4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]
tm4['1st Most Common Venue'].value_counts()

Coffee Shop           45
Café                  10
Italian Restaurant     6
Park                   5
Sandwich Place         2
Greek Restaurant       2
Gastropub              1
Bakery                 1
Harbor / Marina        1
Hotel                  1
Pizza Place            1
Restaurant             1
Name: 1st Most Common Venue, dtype: int64

### As can be seen above in the 4th cluster majority of 1st most common venue are Coffee Shop so this cluster can be termed as 'Coffee Shop' neighbourhoods

In [43]:
tm5=toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]
tm5['1st Most Common Venue'].value_counts()

Coffee Shop                  7
Fast Food Restaurant         7
Clothing Store               5
Park                         4
Indian Restaurant            3
Hotel                        2
Middle Eastern Restaurant    2
Pizza Place                  2
Pharmacy                     2
Chinese Restaurant           1
Furniture / Home Store       1
Ice Cream Shop               1
Convenience Store            1
Asian Restaurant             1
Greek Restaurant             1
Bank                         1
Sandwich Place               1
Mediterranean Restaurant     1
Salon / Barbershop           1
Japanese Restaurant          1
Name: 1st Most Common Venue, dtype: int64

### As can be seen above in the 5th cluster the 1st most common venues international cuisine restuarant so this cluster can be termed as 'International Cuisine' neighbourhoods